This was written with Toyplot 0.13. It definitely won't work with older versions.

In [8]:
import numpy
import pandas
import toyplot.pdf

print toyplot.__version__

0.13.0


Read in the data from a csv file.

In [9]:
plot_data = pandas.read_csv('Average_Overall_SpeedUp.csv')
plot_data

,Unnamed: 0,Algorithm,Speedup,Format
0,1,2,1.503795,Explorable
1,2,3,2.858407,Explorable
2,3,4,3.668910,Explorable
3,4,2,2.036321,Static
4,5,3,2.397479,Static
5,6,4,2.824415,Static


Change order of entries.

In [10]:
# Reorder rows based on non-lexigraphical sorting we want
plot_data.Format = plot_data.Format.astype('category')
plot_data.Format.cat.set_categories(['Static','Explorable'], inplace=True)
plot_data.sort_values(['Format', 'Algorithm'], inplace=True)

# Sort columns in the order we want
plot_data = plot_data[['Format', 'Algorithm', 'Speedup']]

plot_data

,Format,Algorithm,Speedup
3,Static,2,2.036321
4,Static,3,2.397479
5,Static,4,2.824415
0,Explorable,2,1.503795
1,Explorable,3,2.858407
2,Explorable,4,3.668910


Change the values to printable names.

In [12]:
plot_data.rename(index=str, columns={'Format': 'Type'}, inplace=True)
plot_data.rename(index=str, columns={'Algorithm': 'Alg'}, inplace=True)
plot_data.rename(index=str, columns={'Speedup': 'Speed-up'}, inplace=True)
plot_data

,Type,Alg,Speed-up
3,Static,2,2.036321
4,Static,3,2.397479
5,Static,4,2.824415
0,Explorable,2,1.503795
1,Explorable,3,2.858407
2,Explorable,4,3.668910


Duplicate value columns to put in bar chart.

In [13]:
plot_data.insert(3, 'Speed-up', plot_data['Speed-up'], allow_duplicates=True)
plot_data

,Type,Alg,Speed-up,Speed-up
3,Static,2,2.036321,2.036321
4,Static,3,2.397479,2.397479
5,Static,4,2.824415,2.824415
0,Explorable,2,1.503795,1.503795
1,Explorable,3,2.858407,2.858407
2,Explorable,4,3.668910,3.668910


In [14]:
max_value = plot_data.max()[2:].max()
max_value

3.6689095101814697

In [18]:
canvas = toyplot.Canvas(width=500, height=225)
table = canvas.table(plot_data)
table.cells.cell[:,2:].format = toyplot.format.FloatFormatter(format='{:.1f}')

# Generally I do not want vertical lines, but they can be handy when resizing columns.
#table.grid.vlines[...] = "single"

table.column(0).width = 70
table.column(1).width = 25
table.column(2).width = 30
table.column(2).column_offset = 8

# Left align the text in cells, center algorithm number
table.body.cell[:, 0:1].align = 'left'
table.body.cell[:, 1].align = 'center'

# Merge header for numbers and bar chart
table.top.cell[0,2:4].merge().data = "Average Speed-up"

# Make bar chart columns
axes = table.body.column[3].cartesian()
axes.cell_bars(width=1)
axes.x.domain.max = max_value

# Add gaps to separate groups
table.body.gaps.rows[[2]] = '3pt'

Type Alg Static 2 2 . 0 Static 3 2 . 4 Static 4 2 . 8 Explorable 2 1 . 5 Explorable 3 2 . 9 Explorable 4 3 . 7 Average Speed-up 
 
 
 Save as .csv

In [19]:
toyplot.pdf.render(canvas, 'Average_Overall_SpeedUp.pdf')